# Evaluación de Modelos para Clasificación Multiclase

Este notebook contiene el desarrollo completo de modelos para clasificar pacientes como:

- `0 = No diabetes`
- `1 = Prediabetes`
- `2 = Diabetes`

Se incluyen:
- Preprocesamiento y escalado
- Rebalanceo con SMOTE y SMOTEENN
- Entrenamiento de modelos: Random Forest, XGBoost, LightGBM
- Pipeline jerárquico
- Evaluación y comparación de resultados


## 🧹 Carga, preprocesamiento y escalado

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Cargar datos
df = pd.read_csv("../data/diabetes_multiclass.csv").drop_duplicates()

# Variables seleccionadas
features = [
    'BMI', 'Age', 'HighBP', 'HighChol', 'GenHlth',
    'PhysHlth', 'DiffWalk', 'HeartDiseaseorAttack'
]

X = df[features]
y = df["Diabetes_012"]

# División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 🔁 Rebalanceo con SMOTE y SMOTEENN

In [2]:
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN

# SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_scaled, y_train)

# SMOTEENN
smoteenn = SMOTEENN(random_state=42)
X_train_smoteenn, y_train_smoteenn = smoteenn.fit_resample(X_train_scaled, y_train)

## 🌲 Random Forest con SMOTEENN

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

rf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [100],
    'max_depth': [10],
    'min_samples_split': [2],
    'class_weight': ['balanced']
}
grid = GridSearchCV(rf, param_grid, cv=3, n_jobs=-1)
grid.fit(X_train_smoteenn, y_train_smoteenn)

y_pred_rf = grid.predict(X_test_scaled)
print(classification_report(y_test, y_pred_rf, digits=3))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf, display_labels=["No", "Prediabetes", "Diabetes"])
plt.title("Random Forest + SMOTEENN")
plt.show()

# 🌲 Modelo: Random Forest + SMOTEENN

## 🔧 Descripción
Modelo de clasificación multiclase usando RandomForestClassifier, entrenado sobre datos rebalanceados con SMOTEENN para equilibrar las clases.

- Algoritmo: Random Forest
- Rebalanceo: ✅ SMOTE + ENN (elimina ruido)
- Hiperparámetros probados:
  - n_estimators: 100
  - max_depth: 10
  - class_weight: balanced

## 📊 Resultados esperados
- Accuracy: ~61%
- Recall clase 1 (Prediabetes): ~15%
- Rendimiento robusto, aunque limitado en recall en clases minoritarias.

## ✅ Observación
Es un buen baseline, especialmente útil si se desea un modelo interpretativo basado en árboles.


## 🚀 XGBoost con SMOTEENN

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(objective='multi:softmax', num_class=3, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
param_grid = {
    'n_estimators': [100],
    'max_depth': [6],
    'learning_rate': [0.1]
}
grid = GridSearchCV(xgb, param_grid, cv=3, n_jobs=-1)
grid.fit(X_train_smoteenn, y_train_smoteenn)

y_pred_xgb = grid.predict(X_test_scaled)
print(classification_report(y_test, y_pred_xgb, digits=3))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_xgb, display_labels=["No", "Prediabetes", "Diabetes"])
plt.title("XGBoost + SMOTEENN")
plt.show()

# 🚀 Modelo: XGBoost + SMOTEENN

## 🔧 Descripción
Modelo de boosting entrenado con XGBoost sobre un conjunto balanceado con SMOTEENN. Se enfoca en precisión y control del error mediante optimización por gradiente.

- Algoritmo: XGBoostClassifier
- Rebalanceo: ✅ SMOTE + ENN
- Hiperparámetros probados:
  - max_depth: 6
  - learning_rate: 0.1
  - n_estimators: 100

## 📊 Resultados esperados
- Recall clase 1 (Prediabetes): ~8%
- F1-score clase 1: ~0.05
- Mejor estabilidad general que RandomForest, pero menor sensibilidad en predicción de clase 1.

## ✅ Observación
Adecuado para pipelines más exigentes en precisión global. Complementario a modelos tipo RandomForest.


## 💡 LightGBM sin rebalanceo (modelo final)

In [1]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='multiclass', num_class=3, class_weight='balanced', random_state=42)
param_grid = {
    'n_estimators': [200],
    'max_depth': [10],
    'learning_rate': [0.3],
    'subsample': [0.8],
    'colsample_bytree': [1.0]
}
grid = GridSearchCV(lgbm, param_grid, cv=3, n_jobs=-1)
grid.fit(X_train_scaled, y_train)

y_pred_lgbm = grid.predict(X_test_scaled)
print(classification_report(y_test, y_pred_lgbm, digits=3))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_lgbm, display_labels=["No", "Prediabetes", "Diabetes"])
plt.title("LightGBM sin rebalanceo")
plt.show()

NameError: name 'GridSearchCV' is not defined

# 💡 Modelo: LightGBM sin rebalanceo (modelo final)

## 🔧 Descripción
LightGBM entrenado directamente sobre datos originales, sin técnicas de oversampling, utilizando `class_weight='balanced'` para manejar el desbalance.

- Algoritmo: LGBMClassifier
- Rebalanceo: ❌ No aplicado
- Escalado: StandardScaler
- Hiperparámetros finales:
  - n_estimators: 200
  - max_depth: 10
  - learning_rate: 0.3
  - subsample: 0.8
  - colsample_bytree: 1.0

## 📊 Resultados clave
- Recall clase 1 (Prediabetes): **23.7%**
- Accuracy total: **61.6%**
- Recall clase 2 (Diabetes): 60.4%

## ✅ Conclusión
Este modelo proporciona el mejor balance general sin necesidad de rebalanceo artificial, y es el **modelo final seleccionado para producción**.


## 🧾 Conclusiones

- **Random Forest + SMOTEENN**: buen rendimiento general pero bajo recall en clase 1.
- **XGBoost + SMOTEENN**: similar a RF pero con menor precisión en clase 1.
- **LightGBM sin rebalanceo**: modelo más equilibrado, con mejor recall en clase 1 (prediabetes).

👉 Se selecciona **LightGBM multiclase sin rebalanceo** como modelo final por simplicidad, rendimiento y estabilidad.


# 🧠 Model Card - LightGBM Multiclase Final

## 📋 Descripción del Modelo

Este modelo utiliza **LightGBM** para predecir si un paciente pertenece a una de tres clases:

- `0 = No diabetes`
- `1 = Prediabetes`
- `2 = Diabetes`

Ha sido entrenado **sin técnicas de rebalanceo** artificial, confiando únicamente en `class_weight='balanced'` para manejar el desbalance.

## 🧪 Entrenamiento

- Algoritmo: `LGBMClassifier`
- Tipo: Multiclase (3 clases)
- Dataset: `CDC Diabetes Health Indicators`
- Features utilizadas:
  - BMI, Age, HighBP, HighChol, GenHlth, PhysHlth, DiffWalk, HeartDiseaseorAttack
- Rebalanceo: ❌ No aplicado
- Escalado: `StandardScaler`

## ⚙️ Hiperparámetros

```python
{
  'n_estimators': 200,
  'max_depth': 10,
  'learning_rate': 0.3,
  'subsample': 0.8,
  'colsample_bytree': 1.0
}
```

## 🎯 Métricas de Evaluación

| Clase         | Precision | Recall | F1-score |
|---------------|-----------|--------|----------|
| No diabetes   | 0.941     | 0.627  | 0.753    |
| Prediabetes   | 0.026     | 0.237  | 0.048    |
| Diabetes      | 0.344     | 0.604  | 0.438    |

- **Accuracy total**: 61.6%
- **Macro recall**: 48.9%
- **Macro F1-score**: 41.3%

## ✅ Conclusión

Este modelo es el que mejor equilibra simplicidad, rendimiento y generalización, siendo especialmente destacable en la clase más difícil (prediabetes), donde supera el 23% de recall sin rebalanceo externo.

Se considera el **modelo final recomendado** para producción e interpretación con SHAP.

